# Close cross Ema - Strategy - Backtesting 

### Import Library

In [106]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)
from backtesting import set_bokeh_output
set_bokeh_output(notebook=False)

### Load Price Data

In [107]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [108]:
data = dataset.copy()

In [109]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [110]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627


In [111]:
def get_signal(r):
    signal = ''
    if r['cross_fline']:
        if r['Open'] < r['Close'] and r['ema_fast'] > r['ema_low']:
            signal = 'long'
        elif r['Open'] > r['Close'] and r['ema_fast'] < r['ema_low']:
            signal = 'short'
    elif r['cross_fline_s1']:
        if r['ema_fast'] > r['ema_low'] and r['Open'] < r['Close'] and r['Open_s1'] > r['Close_s1']:
            signal = 'long'
        elif r['ema_fast'] < r['ema_low'] and r['Open'] > r['Close'] and r['Open_s1'] < r['Close_s1']:
            signal = 'short'
    return signal

In [112]:
def prepare_data(data):
    data['Open_s1'] = data['Open'].shift(1)
    data['Close_s1'] = data['Close'].shift(1)
    data['max_5'] = data['High'].rolling(5).max()
    data['min_5'] = data['Low'].rolling(5).min()
    data["ema_fast"] = ta.ema(data["Close"], length=20)
    data["ema_low"] = ta.ema(data["Close"], length=250)
    data['cross_fline'] = data.apply(lambda r: True if (r['Close'] > r['ema_fast'] > r['Open'] or r['Close'] < r['ema_fast'] < r['Open']) else False, axis=1)
    data['cross_fline_s1'] = data['cross_fline'].shift(1)
    data['signal'] = data.apply(lambda r: get_signal(r), axis=1)
    return data

In [113]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [114]:
prepared_data

,Open,High,Low,Close,Volume,Open_s1,Close_s1,max_5,min_5,ema_fast,ema_low,cross_fline,cross_fline_s1,signal
Date,,,,,,,,,,,,,,
2020-11-06 14:10:00,908.0,911.9,907.5,911.0,5068,908.7,907.6,911.9,906.7,907.594675,908.782400,False,False,
2020-11-06 14:15:00,911.0,912.7,910.5,910.6,5948,908.0,911.0,912.7,906.7,907.880896,908.796883,False,False,
2020-11-06 14:20:00,910.7,911.2,909.4,910.9,4466,911.0,910.6,912.7,907.3,908.168430,908.813641,False,False,
2020-11-06 14:25:00,910.5,911.0,908.5,910.0,3602,910.7,910.9,912.7,907.5,908.342865,908.823094,False,False,
2020-11-06 14:30:00,910.5,910.7,910.5,910.7,78,910.5,910.0,912.7,907.5,908.567354,908.838049,False,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-25 14:25:00,1298.0,1298.3,1296.7,1297.0,5524,1297.1,1298.0,1298.7,1295.9,1297.302274,1287.412327,True,True,
2024-11-25 14:30:00,1297.1,1297.1,1297.1,1297.1,161,1298.0,1297.0,1298.7,1295.9,1297.283010,1287.489520,False,True,
2024-11-25 14:45:00,1298.4,1298.4,1298.4,1298.4,5627,1297.1,1297.1,1298.7,1296.2,1297.389390,1287.576456,False,False,


In [115]:
prepared_data[prepared_data.signal != ''].tail(20)

,Open,High,Low,Close,Volume,Open_s1,Close_s1,max_5,min_5,ema_fast,ema_low,cross_fline,cross_fline_s1,signal
Date,,,,,,,,,,,,,,
2024-11-19 14:15:00,1268.5,1268.6,1265.2,1266.7,10698,1268.9,1268.8,1271.8,1264.4,1267.542784,1283.771154,True,False,short
2024-11-20 10:05:00,1261.2,1261.2,1259.5,1259.9,6161,1258.9,1261.2,1261.2,1254.8,1259.797464,1280.398398,False,True,short
2024-11-20 10:10:00,1260.0,1260.8,1259.0,1259.3,5048,1261.2,1259.9,1261.2,1256.3,1259.750087,1280.230283,True,False,short
2024-11-20 10:25:00,1260.2,1260.8,1258.5,1259.0,3999,1259.7,1259.9,1261.2,1258.5,1259.687663,1279.739423,True,True,short
2024-11-20 14:05:00,1277.4,1277.7,1272.1,1273.4,11389,1275.8,1277.5,1279.5,1272.1,1273.977990,1278.441158,True,False,short
2024-11-20 14:15:00,1274.2,1274.7,1269.6,1270.6,11619,1273.4,1274.3,1278.7,1269.6,1273.684023,1278.345944,True,True,short
2024-11-21 11:10:00,1268.5,1268.6,1267.5,1267.8,3224,1269.3,1268.6,1270.5,1267.5,1268.318437,1276.019811,True,False,short
2024-11-22 10:30:00,1292.2,1293.2,1292.0,1293.0,3694,1293.2,1292.0,1296.2,1291.8,1292.181300,1278.744557,False,True,long
2024-11-22 13:50:00,1294.8,1295.7,1293.5,1295.7,8175,1300.0,1295.0,1300.7,1293.5,1296.824788,1281.774693,False,True,long


In [116]:
class MainStrategy(Strategy):
    max_sl = 3.1
    trailing_sl = 4.5
    tp_step = 12
    def init(self):
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        close_price = self.data.Close[-1]
        if self.position.is_long:
            max_5 = self.data.max_5[-1]
            if close_price < max_5 - self.trailing_sl:
                self.position.close()
        elif self.position.is_short:
            min_5 = self.data.min_5[-1]
            if close_price > min_5 + self.trailing_sl:
                self.position.close()
 
        if self.position:
            return  
        signal = self.data.signal[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - self.max_sl
            tp = buy_price + self.tp_step
            self.buy(size=1, sl=sl, tp=tp)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + self.max_sl
            tp = sell_price - self.tp_step
            self.sell(size=1, sl=sl, tp=tp)

In [117]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [118]:
stats

Start                     2020-11-06 14:10:00
End                       2024-11-26 09:05:00
Duration                   1480 days 18:55:00
Exposure Time [%]                   36.081954
Equity Final [$]                   9986.53923
Equity Peak [$]                   10069.18373
Return [%]                          -0.134608
Buy & Hold Return [%]               42.502744
Return (Ann.) [%]                   -0.033503
Volatility (Ann.) [%]                1.065615
Sharpe Ratio                         -0.03144
Sortino Ratio                       -0.047246
Calmar Ratio                        -0.013626
Max. Drawdown [%]                   -2.458797
Avg. Drawdown [%]                   -0.186359
Max. Drawdown Duration     1270 days 23:05:00
Avg. Drawdown Duration       58 days 17:27:00
# Trades                                 1934
Win Rate [%]                        34.281282
Best Trade [%]                       1.317967
Worst Trade [%]                     -0.491069
Avg. Trade [%]                    

In [119]:
stats['_trades'].tail(20)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
1914,-1,51044,51064,1317.60460,1306.0,11.60460,0.008807,2024-11-12 10:15:00,2024-11-12 13:20:00,None,0 days 03:05:00
1915,-1,51072,51073,1307.60760,1309.8,-2.19240,-0.001677,2024-11-12 14:00:00,2024-11-12 14:05:00,None,0 days 00:05:00
1916,-1,51089,51121,1307.90751,1305.0,2.90751,0.002223,2024-11-13 09:45:00,2024-11-13 13:50:00,None,0 days 04:05:00
1917,-1,51134,51143,1303.30889,1306.4,-3.09111,-0.002372,2024-11-14 09:15:00,2024-11-14 10:00:00,None,0 days 00:45:00
1918,-1,51145,51145,1304.00868,1307.5,-3.49132,-0.002677,2024-11-14 10:10:00,2024-11-14 10:10:00,None,0 days 00:00:00
1919,-1,51147,51177,1307.80754,1296.1,11.70754,0.008952,2024-11-14 10:20:00,2024-11-14 14:15:00,None,0 days 03:55:00
1920,-1,51217,51218,1281.21552,1284.7,-3.48448,-0.002720,2024-11-15 13:20:00,2024-11-15 13:25:00,None,0 days 00:05:00
1921,-1,51221,51225,1282.61510,1286.1,-3.48490,-0.002717,2024-11-15 13:40:00,2024-11-15 14:00:00,None,0 days 00:20:00
1922,-1,51227,51231,1280.41576,1280.0,0.41576,0.000325,2024-11-15 14:10:00,2024-11-15 14:30:00,None,0 days 00:20:00
1923,-1,51288,51314,1271.21852,1270.3,0.91852,0.000723,2024-11-19 09:20:00,2024-11-19 11:30:00,None,0 days 02:10:00


## Optimize

In [120]:
max_sl_params = [x / 10.0 for x in range(20, 50, 2)]
trailing_sl_params = [x / 10.0 for x in range(20, 60, 5)]
tp_step_params = [x / 10.0 for x in range(20, 200, 5)]
optimize_stats, optimize_table = bt.optimize(max_sl=max_sl_params,
                                      trailing_sl=trailing_sl_params,
                                      tp_step=tp_step_params,
                    maximize='Return [%]', 
                    
                    random_state=0,
                    return_heatmap=True)

In [121]:
optimize_stats

Start                     2020-11-06 14:10:00
End                       2024-11-26 09:05:00
Duration                   1480 days 18:55:00
Exposure Time [%]                   37.657399
Equity Final [$]                  10158.80512
Equity Peak [$]                   10198.78861
Return [%]                           1.588051
Buy & Hold Return [%]               42.502744
Return (Ann.) [%]                    0.392718
Volatility (Ann.) [%]                1.158206
Sharpe Ratio                         0.339075
Sortino Ratio                        0.566279
Calmar Ratio                         0.177819
Max. Drawdown [%]                   -2.208523
Avg. Drawdown [%]                   -0.105239
Max. Drawdown Duration     1126 days 04:40:00
Avg. Drawdown Duration       19 days 18:44:00
# Trades                                 1909
Win Rate [%]                        32.949188
Best Trade [%]                       1.929322
Worst Trade [%]                     -0.459632
Avg. Trade [%]                    

In [122]:
optimize_table

max_sl  trailing_sl  tp_step
2.0     2.0          2.0       -10.115871
                     2.5        -8.468942
                     3.0        -7.885576
                     3.5        -7.640521
                     4.0        -6.786606
                                  ...    
4.8     5.5          17.5        0.359797
                     18.0        0.496797
                     18.5        0.543797
                     19.0        0.762797
                     19.5        0.926797
Name: Return [%], Length: 4320, dtype: float64